# parse the SBGN file

In [27]:
def sbgn_to_vega(sbgn_file_name, sbgn_vega_template, vega_output_filename, data_url):
    """ Convert a pathway map from SBGN to Vega format.
    Args:
        sbgn_file_name (:obj:`str`): the name of the SBGN file that will be parsed
        sbgn_vega_template (:obj:`str`): path to the template that will interpret the sbgn visual in Vega
        vega_output_filename (:obj:`str`): path to the ginsim simulation data
        data_url (str): path of the data SBGN data
    """          
    # import the libraries and model file
    from math import inf, degrees, atan2
    import xml.etree.ElementTree as ET
    import json
    
    # define the SBGN file 
    root = ET.parse(sbgn_file_name).getroot()
    graph = root.find('{http://sbgn.org/libsbgn/0.2}map')

    # ============================================== PARSE THE NODES ===============================================
    nodes = []
    for child in graph.findall('{http://sbgn.org/libsbgn/0.2}glyph'):
        regulations = {}
        # determine the node process
        node_process = child.attrib['class']

        # determine the node label
        node_label = child.find('{http://sbgn.org/libsbgn/0.2}label')
        if node_label is not None:
            node_label = node_label.attrib['text']

        # determine the node dimensions and coordinates
        node_coordinates = child.find('{http://sbgn.org/libsbgn/0.2}bbox') 
        y_coordinate = node_coordinates.attrib['y']
        x_coordinate = node_coordinates.attrib['x']
        node_width = node_coordinates.attrib['w']
        node_height = node_coordinates.attrib['h']

        # organize the aforementioned data 
        nodes.append({'id': node_label,
                      'process': node_process,
                      'x_value':x_coordinate,
                      'y_value':y_coordinate,
                      'node_width': node_width,
                      'node_height': node_height})

    # ============================================== PARSE THE EDGES ===============================================
    # determine the maximum boundaries of the figure
    min_x = inf
    min_y = inf
    max_x = -inf
    max_y = -inf
    for node in nodes:
        min_x = min(min_x, float(node['x_value']))
        max_x = max(max_x, float(node['x_value']))
        min_y = min(min_y, float(node['y_value']))
        max_y = max(max_y, float(node['y_value']))     

    sbgn_width = max_x - min_x
    sbgn_height = max_y - min_y
    max_width_height = 700

    if sbgn_width > sbgn_height:
        width = max_width_height
        coordinate_scale = float(width / sbgn_width)
        height = sbgn_height * coordinate_scale
    else:
        height = max_width_height
        coordinate_scale = float(height / sbgn_height)
        width = sbgn_width * coordinate_scale

    width += 200 
    height += 50 

    # parse the edges
    edges = []
    edge_ends = []
    for child in graph.findall('{http://sbgn.org/libsbgn/0.2}arc'):
        # determine the edge process and general information
        edge_id = child.attrib['id']
        to_node = child.attrib['target']
        from_node = child.attrib['source']

        # determine the coordinates of the edge
        net_x_adjustment = 0
        net_y_adjustment = 0

        initial_point = child.find('{http://sbgn.org/libsbgn/0.2}start') 
        x1 = initial_point.attrib['x']
        y1 = initial_point.attrib['y']
        x1 = (float(x1) + net_x_adjustment) * coordinate_scale
        y1 = (float(y1) + net_y_adjustment) * coordinate_scale

        end_point = child.find('{http://sbgn.org/libsbgn/0.2}end') 
        x2 = end_point.attrib['x']
        y2 = end_point.attrib['y']
        x2 = (float(x2) + net_x_adjustment) * coordinate_scale
        y2 = (float(y2) + net_y_adjustment) * coordinate_scale

        delta_x = x2 - x1
        delta_y = y2 - y1

        # create the SVG path
        edge_svg_path = 'M {} {}'.format(x1, y1)
        if abs(delta_x) < abs(delta_y):
            edge_svg_path += ' v {}'.format(delta_y)
            delta_x = 0
        elif abs(delta_x) > abs(delta_y):
            edge_svg_path += ' h {}'.format(delta_x)
            delta_y = 0
        else:
            print('ERROR: The {} edge is not orthogonal, and is omitted.'.format(edge_id))
            continue 

        # assign the calculated values to a list of dictionaries
        edges.append({
            "id": edge_id,
            'edge_svg_path': edge_svg_path,
            'related_nodes': [from_node, to_node]
        })

        # assign edge ends to the figure
        edge_process = child.attrib['class']
        end_fill = 'black'
        stroke_width = 2
        if edge_process == 'inhibition':
            edge_end_shape = 'stroke'
            edge_end_angle = degrees(atan2(delta_y, delta_x)) + 90
            stroke_width = 5
        else:
            edge_end_shape = 'triangle'
            edge_end_angle = degrees(atan2(delta_y, delta_x))
            if edge_process == 'necessary stimulation':
                end_fill = 'white'

        # adjusting the figure proportions
        general_node_width = 0
        general_node_height = 0
        for node in nodes:
            if node['id'] == to_node:
                general_node_width = node['node_width']
                general_node_height = node['node_height']    
                break
        if general_node_width == 0:
            print('ERROR: The dimensions of the {} node are not represented.'.format(to_node))

        edge_ends_x_offset = 0
        edge_ends_y_offset = 0

        if delta_x > 0:
            edge_end_x_buffer = -general_node_width
            edge_end_y_buffer = 0
            if edge_end_shape == 'triangle':
                edge_end_angle -= 150
        elif delta_x < 0:
            edge_end_x_buffer = general_node_width
            edge_end_y_buffer = 0
            if edge_end_shape == 'triangle':
                edge_end_angle += 210
        if delta_y > 0:
            edge_end_x_buffer = 0
            edge_end_y_buffer = -general_node_height
            if edge_end_shape == 'triangle':
                edge_end_angle += 90
        elif delta_y < 0:
            edge_end_x_buffer = 0
            edge_end_y_buffer = general_node_height
            if edge_end_shape == 'triangle':
                edge_end_angle += 90

        # organizing the edge end information
        edge_ends.append({
            "id": edge_id,
            'x': x2 + edge_ends_x_offset,
            'y': y2 + edge_ends_y_offset,
            'edge_end_angle': edge_end_angle,
            'edge_end_shape': edge_end_shape,
            'stroke_width': stroke_width,
            'edge_end_fill': end_fill
        })


    # ============================================== EXPORT TO VEGA ===============================================
    # import the Vega template
    vega = json.load(open(sbgn_vega_template))
    signal_height = 100
    vega['width'] = width
    vega['height'] = height + signal_height

    # assign values to the Vega file
    for entry in vega['data']:
        name = entry['name']
        if name == 'nodesData':
            entry['values'] = nodes
        elif name == 'edgesData':
            entry['values'] = edges
        elif name == 'edgeEndCoordinatesData':
            entry['values'] = edge_ends

    for signal in vega['signals']:
        name = signal['name']
        signal_value = signal['value']

        if name == 'edgeEndStrokeWidth':
            signal_value = 1.5 * coordinate_scale
        elif name == 'signalHeight':
            signal_value = signal_height
        elif name == 'signalPadding':
            signal_value = 0
        elif name == 'nodeStrokeWidthData':
            signal_value = 1 * coordinate_scale
        elif name == 'edgeStrokeWidthData':
            signal_value = 18 * coordinate_scale
        elif name == 'mapMaxX':
            signal_value = width
        elif name == 'mapMaxY':
            signal_value = height

    # export Vega SBGN visualization
    with open(vega_output_filename, 'w') as file:
        json.dump(vega, file, indent = 4)            

# Parse the Repressilator_PD_v6 example

In [29]:
# define the arguments
sbgn_file_name = 'Repressilator_PD_v6_color.sbgn'
sbgn_vega_template = 'sbgn_to_vega.template.json'
vega_output_filename = 'SBGN_repressilator-v6_in_vega_1.json'
data_url = 'https://run.api.biosimulations.org/results/60a1a1754c88d864b0aa4dd2/simulation.sedml%2Freport_wt?sparse=false'

# execute the example through sbgn_to_vega()
sbgn_to_vega(sbgn_file_name, sbgn_vega_template, vega_output_filename, data_url)

ERROR: The dimensions of the glyph4.2 node are not represented.
ERROR: The dimensions of the glyph2 node are not represented.
ERROR: The dimensions of the glyph7.2 node are not represented.
ERROR: The dimensions of the glyph12 node are not represented.
ERROR: The dimensions of the glyph5.2 node are not represented.
ERROR: The dimensions of the glyph13.1 node are not represented.
ERROR: The dimensions of the glyph8 node are not represented.
ERROR: The dimensions of the glyph15 node are not represented.
ERROR: The dimensions of the glyph27 node are not represented.
ERROR: The dimensions of the glyph26.2 node are not represented.
ERROR: The dimensions of the glyph16.2 node are not represented.
ERROR: The dimensions of the glyph32 node are not represented.
ERROR: The dimensions of the glyph30 node are not represented.
ERROR: The dimensions of the glyph14.1 node are not represented.
ERROR: The dimensions of the glyph23.1 node are not represented.
ERROR: The dimensions of the glyph24 node ar

# import the SVG file

In [ ]:

root = ET.parse('Repressilator_PD_v6_color.svg')


# indent the SVG file

In [ ]:
import re
from svgutils import transform, compose
from IPython.display import SVG

svg = transform.fromfile('Repressilator_PD_v6_color.svg')
print(svg.get_size())
SVG(svg)

'''"replacerules.rules": {
    "svg-path": {
      "find": ["(?<!^ +)([MmLlHhVvCcSsQqTtAaZz]) *","(?<=^ +)([MmLlHhVvCcSsQqTtAaZz]) *"],
      "replace": ["\n  $1 ","$1 "]
    }
  }'''